### KRX 데이터정보시스템
- 매일 주식거래 내용

['일자', '종가', '등락률', '시가', '고가', '저가', '거래대금'] 등 수집

In [28]:
from selenium import webdriver as wd
from selenium.webdriver import ActionChains # scroll down 사용하기 위하여 선서
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs

import datetime, time
from datetime import date

import pandas as pd
import numpy as np
import requests
import time
import os, sys

In [29]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [30]:
# !pip install chromedriver_autoinstaller

import chromedriver_autoinstaller

driver_name = chromedriver_autoinstaller.install()  # Check if the current version of chromedriver exists
                                      # and if it doesn't exist, download it automatically,
                                      # then add chromedriver to path
time.sleep(1)

In [31]:
driver = wd.Chrome(driver_name)

In [32]:
# driver.set_window_position(-10000,0) # hide windows
main_url = 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020103'
# 개별종목 시세추이 data-menu-id: MDC0201020103
driver.get(main_url)
time.sleep(1)

In [33]:
company_name = ['005930/삼성전자', '005380/현대차', '035420/NAVER', '033780/KT&G']
company_code = {'sec': '005930', 'hyunmotor': '005380', 'naver': '035420', 'ktng': '033780'}
# company_name = ['005380/현대차']
# company_code = {'hyunmotor': '005380'}

company_match = {'005930': '005930/삼성전자', '005380': '005380/현대차',
                 '035420': '035420/NAVER', '033780': '033780/KT&G'}

In [34]:
print("최근 몇일전까지 자료를 추가할까요?, ", end='')
n_days = int(input().strip())
today = datetime.date.today()
diff_days = datetime.timedelta(days=n_days) # 최근 n_days일 이내 데이터 취득
start_date = today - diff_days
end_date = today

최근 몇일전까지 자료를 추가할까요?, 20


In [35]:
# 백만원 단위 표시 선정
css_sel = '#MDCSTAT017_FORM > div.CI-MDI-UNIT-WRAP > div > p:nth-child(2) \
           > select.CI-MDI-UNIT-MONEY > option:nth-child(3)'
driver.find_element_by_css_selector(css_sel).click()
time.sleep(1)

In [36]:
def get_data_company(com_name, start_date, end_date):

    # 회사이름 입력 Q 버튼
    driver.find_element_by_css_selector('#btnisuCd_finder_stkisu0_0').click()
    time.sleep(2)

    # pop up된 입력창에서 회사이름 입력
    driver.find_element_by_id('searchText__finder_stkisu0_0').clear()
    time.sleep(1)
    driver.find_element_by_id('searchText__finder_stkisu0_0').send_keys(com_name)
    time.sleep(2)

    # 검색 버튼 푸시
    driver.find_element_by_css_selector('#searchBtn__finder_stkisu0_0').click()
    time.sleep(2)

    # 테이블에서 최종 선택
    css_sel = '#jsGrid__finder_stkisu0_0 > tbody > tr:nth-child(1) > td:nth-child(1)'
    driver.find_element_by_css_selector(css_sel).click()
    time.sleep(2)

    return

In [37]:
def set_date(start_date, end_date): # 일정 기간 데이터 취득
    # end_date를 먼저 입력하고 start date 입력. 반대로 하면 start date가 이전날짜로  reset되어짐
    driver.find_element_by_id('endDd').clear()
    driver.find_element_by_id('endDd').send_keys(end_date)
    time.sleep(1)
    driver.find_element_by_id('strdDd').clear()
    driver.find_element_by_id('strdDd').send_keys(start_date)
    time.sleep(1)

In [38]:
def get_data(start_date, end_date):
    
    column_name = ['date', 'close', 'change', 'close_cr', 'open', 'high', 'low', 
                  'vol_quantity', 'vol','total_amount', 'total_counts' ]
    # ['일자', '종가', '대비', '등락률', '시가', '고가', '저가', '거래량', 
    #                                 '거래대금', '시가총액', '상장주식수']
 
    start_str = start_date.strftime('%Y-%m-%d')
    ed_str = end_date.strftime('%Y-%m-%d')
    set_date(start_str, ed_str)

    # 테이블  취득 버튼 클릭 (우상귀)
    driver.find_element_by_id('jsSearchButton').click()
    time.sleep(5)

    df = pd.read_html(driver.page_source, 
                          attrs={"class": "CI-GRID-BODY-TABLE"}, flavor=["lxml", "bs4"])[0]
    df.columns = column_name
    df['date'] = df['date'].apply(lambda x : datetime.datetime.strptime(x, "%Y/%m/%d"))
    df_get = df[['date', 'close', 'close_cr', 'open', 'high', 'low', 'vol']]
    
    return df_get

In [39]:
def non_empty_index_df(df_input, start_date, end_date): # 토,일,공휴일등 거래가 없는 일자도 모두 포함
    date_range_ts = pd.date_range(start=start_date, end=end_date)
    df_input.set_index('date', inplace=True)
    df_out = pd.DataFrame(columns = df_input.columns)
    df_out.insert(0, 'date', date_range_ts)
    df_out.set_index('date', inplace=True)
    df_out.update(df_input)
    df_out.reset_index(inplace=True)
    return df_out

In [40]:
def append_data(company_code, company_match, com_df):
    for key, val in company_code.items():
        filename = 'data/{}_{}.csv'.format('data', key)
        df_company = pd.read_csv(filename)
        df_company['date'] = df_company['date'].apply(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d"))
        df_company.set_index('date', inplace=True)

        df_new = com_df[company_match[val]]
        df_new.set_index('date', inplace=True)     
        
    #     df_company.update(df_new, overwrite=False)
        df_company.update(df_new)
        df_company.reset_index(inplace=True)
        df_company.to_csv(filename, index=False)
    return df_company

In [41]:
# 메인 데이터 취득
com_df = {}

for com_name in company_name:
    get_data_company(com_name, start_date, end_date)
    df_get = get_data(start_date, end_date)
    df_out = non_empty_index_df(df_get, start_date, end_date)
    com_df[com_name] = df_out.replace(np.nan, '') 
    # NaN to null string. cause NaN does not replace original values in the original datafram
#     com_df[com_name] = get_data()
    
df = append_data(company_code, company_match, com_df)

### 투자자별 현황
 - 거래 정보와 코딩 구성등 형태는 똑 같으나, empty_datetime index 내용과, column selected 내용이 서로 달라서,
    별개의 루틴으로 구성함.

In [42]:
# 투자자별 URL로 변경
main_url = 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020203'
driver.get(main_url)
time.sleep(2)

In [43]:
column_name = ['financial', 'insurance', 'invtrust', 'privequity', 'bank', 'financeetc', 'pension',
              'institution', 'corporateetc', 'retail', 'foreigner', 'foreigneretc', 'total' ]

In [44]:
# Message: element not interactable Error 방지용. 클릭하기 위하여는 그 위치가 클릭할 수 있게 노출되어 있어야 함
# 투자자별 거래실적 버튼이 위치한 곳으로 화면 scroll 

# id가 jsOpenView_1 인 element 를 찾음
stop_tag = driver.find_element_by_id('jsOpenView_1')

# jsOpenView_1 element 까지 스크롤
action = ActionChains(driver)
action.move_to_element(stop_tag).perform()

In [45]:
# 투자자별 거래 실적 버튼 클릭
driver.find_element_by_id('jsOpenView_1').click()
time.sleep(2)

In [46]:
# 백만원 단위 표시 선정
css_sel = '#MDCSTAT023_FORM > div.CI-MDI-UNIT-WRAP > div > p:nth-child(2) > select.CI-MDI-UNIT-MONEY > option:nth-child(3)'
driver.find_element_by_css_selector(css_sel).click()
time.sleep(1)

In [47]:
def date_set(datei): # 하루 하루 데이터를 받아야 함.
# end_date를 먼저 입력하고 start date 입력. 반대로 하면 start date가 이전날짜로  reset되어짐
    driver.find_element_by_id('endDd').clear()
    driver.find_element_by_id('endDd').send_keys(datei)
    time.sleep(1)
    driver.find_element_by_id('strtDd').clear()
    driver.find_element_by_id('strtDd').send_keys(datei)
    time.sleep(1)
    return

In [48]:
def push_button_1(): # 조회 button push
    xp = '/html/body/div[2]/section[2]/section/section/div/div[2]/form/div[1]/div/a' 
    # use full xpath to avoid 'Message: element not interactable' Error
    driver.find_element_by_xpath(xp).click()
    time.sleep(1) # 여유시간 배분
    css_sel = 'div.loading-bar-wrap.small' # 각기 다른 loading 페이지에서 공통적으로 사용됨
    element = WebDriverWait(driver, 20).until(EC.invisibility_of_element_located((By.CSS_SELECTOR, css_sel)))
    # 위 라인은 pop up 창이 사라질 때까지 기다리게 해 줌
    time.sleep(1) # 여유시간 배분
    return

In [49]:
index_name = ['investor', 'sell_quantity', 'buy_quantity', 'pure_buy_quantiry', 'sell', 'buy', 'pure_buy']
def get_daily_data(date_range):
    df_org = None
    for datei in date_range:
        date_set(datei)
        push_button_1()
        df = pd.read_html(driver.page_source, 
                          attrs={"class": "CI-GRID-BODY-TABLE"}, flavor=["lxml", "bs4"])[0]
        df.columns = index_name
        df_new = df[['investor', 'pure_buy']]
        df_new.set_index('investor', inplace=True)
        dft = df_new.T
        dft.columns = column_name
        dft.insert(0, "date", datetime.datetime.strptime(datei, "%Y%m%d"))
        dft.reset_index(drop=True, inplace=True)
        if df_org is None:
            df_org = dft.copy()
            continue
        df_org = df_org.append(dft, ignore_index=True)
        
    return df_org

In [50]:
def get_data_company_investor(com_name, start_date, end_date):
    
    # 회사이름 입력 Q 버튼
    driver.find_element_by_css_selector('#btnisuCd_finder_stkisu0_1').click()
    time.sleep(2)

    # pop up된 입력창에서 회사이름 입력
    driver.find_element_by_id('searchText__finder_stkisu0_1').clear()
    time.sleep(1)
    driver.find_element_by_id('searchText__finder_stkisu0_1').send_keys(com_name)
    time.sleep(2)

    # 검색 버튼 푸시
    driver.find_element_by_css_selector('#searchBtn__finder_stkisu0_1').click()
    time.sleep(2)

    # 테이블에서 최종 선택
    css_sel = '#jsGrid__finder_stkisu0_1 > tbody > tr:nth-child(1) > td:nth-child(1)'
    driver.find_element_by_css_selector(css_sel).click()
    time.sleep(2)
    
    date_range = convert_date(start_date, end_date)
    return get_daily_data(date_range)

In [51]:
def convert_date(start_date, end_date):
    date_range_ts = pd.date_range(start=start_date, end=end_date)
    date_range = []
    for x in date_range_ts:
        date_range.append(datetime.datetime.strftime(x, "%Y%m%d"))
    return date_range

In [52]:
def append_data_investor(company_code, company_match, com_df):
    for key, val in company_code.items():
        filename = 'data/{}_{}.csv'.format('data', key)
        df_company = pd.read_csv(filename)
        df_company['date'] = df_company['date'].apply(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d"))
        df_company.set_index('date', inplace=True)

        df = com_df[company_match[val]]
        df_select = df[['date', 'financial', 'insurance', 'invtrust', 'privequity', 'bank', 'financeetc', 
                        'pension', 'institution', 'corporateetc', 'retail', 'foreigner', 'foreigneretc' ]]
        df_select.set_index('date', inplace=True)
        df_company.update(df_select)
        df_company.reset_index(inplace=True)
        df_company.to_csv(filename, index=False)
    return df_company

In [53]:
# 투자자별 메인 데이터 취득
com_df = {}

for com_name in company_name:
    df_out = get_data_company_investor(com_name, start_date, end_date)
#     com_df[com_name] = df_out.replace(np.nan, '') 
    com_df[com_name] = df_out
    
# 데이터 파일로 저장하기
df_inv = append_data_investor(company_code, company_match, com_df)

In [54]:
driver.close()
driver.quit()